<a href="https://colab.research.google.com/github/shaaagri/iat481-cv-proj/blob/wip/DatasetPrep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook is based on Week 6 tutorial, we use it to check, enumerate, analyse our datasets in terms of correct labeling and balance, before merging them and doing the training with YOLO.

Prerequisites:

In [4]:
from google.colab import drive
drive.mount("/content/MyDrive")

Mounted at /content/MyDrive


The labels for this dataset are stored in YOLO format. i.e.

    class_id bbox_x_center bbox_y_center bbox_width bbox_height

Our pest animals classes:

In [5]:
class_labels = {
    0: "bear",
    1: "raccoon",
    2: "rat",
    3: "skunk"
}

Import the required libraries:

In [87]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random as random
import os

Defining paths to the collected datasets, both found and the ones we prepared ourselves:

In [46]:
dataset_paths = {
  "found_bear": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Bear/Bear.v1i.yolov8",
  "found_raccoon": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Raccoon/Raccoon.v38-416x416-resize.yolov8",
  "found_rat": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Rat/cc-object-detection.v6i.yolov8",
  "found_skunk": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Found/Skunk/Skunk.v1i.yolov8",
  "ours_bear": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Ours/Bear",
  "ours_raccoon": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Ours/Raccoon",
  "ours_rat": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Ours/Rat",
  "ours_skunk": "/content/MyDrive/MyDrive/IAT481/481 CV Project/Datasets/Ours/Skunk",
}

In [91]:
# Collect all image and label files we've got and save into one data structure

def enumerate_dataset(dataset_id, dataset_path):
  image_paths, label_paths = {}, {}

  try:
    image_paths['train'] = [os.path.join(dataset_path, "train", "images", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "train", "images"))]
    image_paths['val'] = [os.path.join(dataset_path, "valid", "images", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "valid", "images"))]
    image_paths['test'] = [os.path.join(dataset_path, "test", "images", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "test", "images"))]
  except FileNotFoundError:
    pass

  try:
    label_paths['train'] = [os.path.join(dataset_path, "train", "labels", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "train", "labels"))]
    label_paths['val'] = [os.path.join(dataset_path, "valid", "labels", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "valid", "labels"))]
    label_paths['test'] = [os.path.join(dataset_path, "test", "labels", image_filename) for image_filename in os.listdir(os.path.join(dataset_path, "test", "labels"))]
  except FileNotFoundError:
    pass

  return image_paths, label_paths

def enumerate_dataset_multi(dataset_paths):
  return {dataset_id: enumerate_dataset(dataset_id, dataset_paths[dataset_id]) for dataset_id in dataset_paths}

In [92]:
datasets = enumerate_dataset_multi(dataset_paths)

In [150]:
# Doing our best with the totality of data we've got to make sure our classes are balanced

dataset_ids = dataset_paths.keys()
merged_dataset = {}

for c in class_labels.values():
  for id in dataset_ids:
    if c in id:
      # create a place in the dictionary for the next class
      if c not in merged_dataset:
        merged_dataset[c] = {}
        merged_dataset[c]['images'] = {}
        merged_dataset[c]['images']['train'] = []
        merged_dataset[c]['images']['val'] = []
        merged_dataset[c]['images']['test'] = []
        merged_dataset[c]['labels'] = {}
        merged_dataset[c]['labels']['train'] = []
        merged_dataset[c]['labels']['val'] = []
        merged_dataset[c]['labels']['test'] = []

      # ignoring KeyError, as some datasets may have only a 'train' folder, but not the others
      try:
        merged_dataset[c]['images']['train'] += datasets[id][0]['train']
        merged_dataset[c]['labels']['train'] += datasets[id][1]['train']
      except KeyError:
        pass

      try:
        merged_dataset[c]['images']['val'] += datasets[id][0]['val']
        merged_dataset[c]['labels']['val'] += datasets[id][1]['val']
      except KeyError:
        pass

      try:
        merged_dataset[c]['images']['test'] += datasets[id][0]['test']
        merged_dataset[c]['labels']['test'] += datasets[id][1]['test']
      except KeyError:
        pass


for k, v in merged_dataset.items():
  print(k, len(v['images']['train']), len(v['images']['val']), len(v['images']['test']))


#file_counts = [len(datasets[id][0]['train']) for id in dataset_ids]

#np.array(file_counts)
#merged_dataset

bear 1500 92 94
raccoon 170 29 17
rat 162 41 20
skunk 220 0 0
bear 1500 92 94
raccoon 170 29 17
rat 162 41 20
skunk 220 0 0
